# Scenario Description 


There exists a lot real-world autonomous driving datasets, such as Waymo and nuPlan.

Those datasets contain rich information about road map, traffic lights, traffic participants and their motions.

Those data can be imported to the MetaDrive so that we can reconstruct an interactive environments and replay (or even alter!) those scenarios recorded in the real world.

To achieve that, we have a separated project called ScenarioNet (https://metadriverse.github.io/scenarionet/) that converts various sources of AD data into a unified data structure, manage those data, and import those data into MetaDrive for creating interactive environments.

As you can see in the next figure of ScenarioNet, the **Unified Scenario Description** is the key for the story.

 
<img src="figs/scenarionet.png" width="800">
